In [7]:
import numpy as np
import math
    
def bin_to_float(bitstring):

    if len(bitstring) != 16 or not all(digit in '01' for digit in bitstring):
        raise ValueError("Invalid bitstring length")
    
    int_val = int(bitstring, 2)
    bytes_val = int_val.to_bytes(2, byteorder='big')

    return np.frombuffer(bytes_val, dtype='f2')[0]
    
print(bin_to_float('0000000000000001'))

1.526e-05


In [8]:
from copy import deepcopy

class Qubit():

    def __init__(self, measurement):
        self.measurement = measurement
        self.alpha = None
        self.beta = None
        self.superposition = False
        
    def apply_hadamard(self):
        superposition_amplitude = 1 / np.sqrt(2)

        self.alpha = superposition_amplitude
        self.beta = superposition_amplitude
        self.superposition = True
        
        return self
        
        
    def apply_xgate(self):
        self.measurement = not self.measurement
        
    def measure(self):
        if not self.superposition:
            return self.measurement
        
        random_var = np.random.uniform(0, 1)
        self.measurement = np.pow(self.beta, 2) >= random_var
        self.superposition = False
        
        return self.measurement
    
    def __str__(self):
        return f'Qubit[superposition: {self.superposition}, [measurement: {self.measurement}]]'

def single_qubit_measurement(qubit):
    return qubit.measure()

def full_measurement(quantum_chromosome, qubits_num):

    for gene in quantum_chromosome:
        gene.measure()
        
    return ''.join(map(lambda x: str(int(x.measure())), quantum_chromosome))

def crossover(q, m, a):
    q_0 = []
    for qubit in q:
        new_qubit = Qubit(False)
        new_qubit.alpha = qubit.alpha
        new_qubit.beta = qubit.beta
        new_qubit.measurement = qubit.measurement
        new_qubit.superposition = qubit.superposition
        q_0.append(new_qubit)
        
    r = np.random.randint(0, len(q) - a)
    
    for k in range(r, r + a):
        if q[k].measurement == m[k].measurement:
            q_0[k].apply_hadamard()
        else:
            q_0[k].apply_xgate()
    
    c_new = full_measurement(q_0, len(q_0))
    
    return c_new

def fitness_first(x):
    return (x - 15) ** 2 

def fitness_second(x):

    if not -5 <= x <= 5:
        return float('inf')  # Penalize out-of-bounds
    
    A = 10
    rastrigin = A + (x**2 - A * math.cos(2 * math.pi * x))
    discontinuity = 50 if -0.1 < x < 0.1 else 0
    noise = np.random.uniform(-0.5, 0.5)
    
    return rastrigin + discontinuity + noise

def deceptive_function(x):

    if not -10 <= x <= 10:
        return float('inf')  # Penalize out-of-bounds
    
    schwefel = 418.9829 - x * math.sin(math.sqrt(abs(x)))
    
    plateau = 100 if -2 <= x <= 2 else 0
    dip = -50 if -0.1 < x < 0.1 else 0
    poly = 0.1 * x**4 - 2 * x**2
    
    return schwefel + plateau + dip + poly

def evolution(population_size, fitness, qubits_in_indiv = 16 ,num_males = 20, num_elites = 20, max_iteration = 500, crossover_size = 3):
        
    population = []
    for i in range(population_size):
        q_0 = [Qubit(False) for _ in range(qubits_in_indiv)]
        q_0 = [qubit.apply_hadamard() for qubit in q_0]
        q = [str(int(qubit.measure())) for qubit in q_0]
        c_i = ''.join(q)
        population.append(c_i)
        
    population.sort(key = lambda x: fitness(bin_to_float(x))) # descending order if maximize 
    
    for indiv in population:
        print(bin_to_float(indiv))
    
    queen = population[0]
    males = population[1: num_males ]
    remaining = population[num_males:]
    # elite = population[:num_elites]
    for _ in range(max_iteration):
        for l in range(population_size - num_elites, population_size):
            population[l] = crossover([Qubit(bool(char)) for char in queen], [Qubit(bool(char)) for char in males[np.random.randint(0, num_males - 1)]], crossover_size)
            
        population.sort(key = lambda x: fitness(bin_to_float(x))) # descending order if maximize 
        queen = population[0]
        males = population[1: num_males ]
        print(f'Iteration: {_}, Queen: {bin_to_float(queen)}, Fitness: {fitness(bin_to_float(queen))}')

    return bin_to_float(queen)
    

evolution(1000, fitness=deceptive_function)

3.201
3.064
3.105
3.055
3.14
3.521
2.863
2.791
2.629
2.459
2.455
2.432
3.938
3.969
2.303
2.102
2.102
2.12
4.16
2.045
4.152
-3.03
-2.781
-2.912
-3.25
-2.688
-3.426
-3.35
-2.588
-2.707
-3.484
-3.572
-2.262
-4.09
4.6
4.73
-4.836
-4.938
-5.035
-5.523
-5.594
0.04068
-0.002096
-0.01999
-0.02046
0.006157
0.09906
-0.0023
5.12e-05
0.001509
-0.007812
8.106e-05
0.05914
-0.03464
-0.0441
0.00639
-0.01109
5.2e-06
0.05847
8.3e-05
-0.0001962
-6.05e-05
0.0008144
-0.02856
-0.02058
-0.01837
0.08417
-0.0007534
8.87e-05
-0.0001866
5.937e-05
-0.011024
-0.003094
0.0003722
0.001378
0.003023
-0.0007195
-0.0002313
0.02443
-0.00875
-0.001639
-0.0003982
0.001562
-0.002794
-0.0171
-0.0461
-0.0004578
-0.01526
-0.0003793
0.0798
-0.0004346
-0.00428
-0.01209
0.0004413
-0.00464
9.716e-05
0.001892
0.0002575
-0.014946
-0.00397
0.0007486
0.004074
0.04556
-0.001398
-0.09076
0.00483
-0.001095
0.05316
0.002346
-0.0001429
-0.0002329
0.002731
0.0005136
0.0010805
0.001849
-0.001963
0.003502
-0.000378
0.009186
0.00814
-0.02785
-

np.float16(3.201)